In [3]:
!pip install kaggle

  Created wheel for kaggle: filename=kaggle-1.5.13-py3-none-any.whl size=77726 sha256=c9ed7762c6e7fdbc8ad4dbb2d8ed0d3efdd6fa8f643c19c9049c2295110d402a
  Stored in directory: c:\users\gowrich1\appdata\local\pip\cache\wheels\fd\97\a6\3372cb23468915cbcf108338dd29c73379fd1a55828ec608ba
Successfully built kaggle


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import requests

In [5]:
!kaggle competitions download -c tweet-sentiment-extraction

403 - Forbidden


In [ ]:
def train(train_data, output_dir, n_iter=20, model=None):
    """Load the model, set up the pipeline and train the entity recognizer."""
    ""
    if model is not None:
        nlp = spacy.load(output_dir)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
    
    # add labels
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        if model is None:
            nlp.begin_training()
        else:
            nlp.resume_training()


        for itn in tqdm(range(n_iter)):
            random.shuffle(train_data)
            batches = minibatch(train_data, size=compounding(4.0, 500.0, 1.001))    
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts,  # batch of texts
                            annotations,  # batch of annotations
                            drop=0.5,   # dropout - make it harder to memorise data
                            losses=losses, 
                            )
            print("Losses", losses)
    save_model(output_dir, nlp, 'st_ner')

In [31]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

np.random.seed(42)
X = np.random.rand(100,1)
y = 2+3*X+np.random.rand(100,1)

X_train,X_test = X[:80],X[80:]
y_train,y_test = y[:80],y[80:]

model = LinearRegression()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

mean_squared_error(y_test,y_pred)

0.092023974529602

In [21]:
X

array([[0.84357158],
       [0.80086261],
       [0.78495556],
       [0.08696996],
       [0.84082567],
       [0.41279926],
       [0.04386086],
       [0.88030337],
       [0.48789875],
       [0.54160418],
       [0.32015214],
       [0.04727444],
       [0.6801233 ],
       [0.29006694],
       [0.77577085],
       [0.21260576],
       [0.78441682],
       [0.3253579 ],
       [0.40196595],
       [0.03668065],
       [0.7369982 ],
       [0.32596835],
       [0.05596522],
       [0.38977007],
       [0.51771671],
       [0.29934933],
       [0.68974291],
       [0.81111467],
       [0.82811795],
       [0.80899023],
       [0.34858756],
       [0.00720637],
       [0.8655517 ],
       [0.87230982],
       [0.25415793],
       [0.21306732],
       [0.67014135],
       [0.36237434],
       [0.88857964],
       [0.27413745],
       [0.47747252],
       [0.38742027],
       [0.67849349],
       [0.84426592],
       [0.97893519],
       [0.43559642],
       [0.02167822],
       [0.793

In [22]:
y

array([[5.3258499 ],
       [5.11615233],
       [4.68555232],
       [3.1931703 ],
       [5.289943  ],
       [4.12445111],
       [2.80531525],
       [4.78076863],
       [3.60608261],
       [4.24459377],
       [3.47075473],
       [3.03507694],
       [4.42469843],
       [3.84515447],
       [4.96512466],
       [3.54592872],
       [4.81963066],
       [3.1779101 ],
       [3.93427751],
       [2.71788281],
       [5.09416963],
       [3.88722255],
       [2.74219695],
       [3.3159981 ],
       [3.69715067],
       [3.21166645],
       [4.26266462],
       [4.87067715],
       [5.28378928],
       [5.28486727],
       [3.33750469],
       [2.4084007 ],
       [5.14883552],
       [4.86757823],
       [3.37988954],
       [3.51215021],
       [4.27663603],
       [3.99331744],
       [5.56845472],
       [3.09090405],
       [4.38702925],
       [3.21474405],
       [4.18612143],
       [4.72568291],
       [5.2119289 ],
       [4.22105963],
       [2.55461573],
       [4.409

In [32]:
import pandas as pd
data = pd.read_csv('Downloads/House Price.csv')
data

No  transaction date  house age  distance to the nearest MRT station  \
0      1          2012.917       32.0                             84.87882   
1      2          2012.917       19.5                            306.59470   
2      3          2013.583       13.3                            561.98450   
3      4          2013.500       13.3                            561.98450   
4      5          2012.833        5.0                            390.56840   
..   ...               ...        ...                                  ...   
409  410          2013.000       13.7                           4082.01500   
410  411          2012.667        5.6                             90.45606   
411  412          2013.250       18.8                            390.96960   
412  413          2013.000        8.1                            104.81010   
413  414          2013.500        6.5                             90.45606   

     number of convenience stores  latitude  longitude  \
0                              10  24.98298  121.54024   
1                               9  24.98034  121.53951   
2                               5  24.98746  121.54391   
3                               5  24.98746  121.54391   
4                               5  24.97937  121.54245   
..                            ...       ...        ...   
409                             0  24.94155  121.50381   
410                             9  24.97433  121.54310   
411                             7  24.97923  121.53986   
412                             5  24.96674  121.54067   
413                             9  24.97433  121.54310   

     house price of unit area  
0                        37.9  
1                        42.2  
2                        47.3  
3                        54.8  
4                        43.1  
..                        ...  
409                      15.4  
410                      50.0  
411                      40.6  
412                      52.5  
413                      63.9  

[414 rows x 8 columns]

In [33]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(data, test_size=0.25, random_state=42)

In [35]:
train

No  transaction date  house age  distance to the nearest MRT station  \
326  327          2013.500        4.1                             56.47425   
410  411          2012.667        5.6                             90.45606   
391  392          2013.583        6.2                           1939.74900   
352  353          2012.833       18.4                           2674.96100   
132  133          2013.167       26.6                            482.75810   
..   ...               ...        ...                                  ...   
71    72          2013.083       35.5                            640.73910   
106  107          2013.083       17.2                            189.51810   
270  271          2013.333       10.8                            252.58220   
348  349          2012.833        4.6                            259.66070   
102  103          2013.083        1.1                            193.58450   

     number of convenience stores  latitude  longitude  \
326                             7  24.95744  121.53711   
410                             9  24.97433  121.54310   
391                             1  24.95155  121.55387   
352                             3  24.96143  121.50827   
132                             5  24.97433  121.53863   
..                            ...       ...        ...   
71                              3  24.97563  121.53715   
106                             8  24.97707  121.54308   
270                             1  24.97460  121.53046   
348                             6  24.97585  121.54516   
102                             6  24.96571  121.54089   

     house price of unit area  
326                      62.1  
410                      50.0  
391                      31.3  
352                      25.7  
132                      37.5  
..                        ...  
71                       40.8  
106                      47.1  
270                     117.5  
348                      53.7  
102                      54.4  

[310 rows x 8 columns]

In [36]:
train.to_csv('HousepriceTrain.csv')
test.to_csv('HousepriceTest.csv')